# Unit 6 - What's the Weather Like?

In this example, you'll be creating a Python script to visualize the weather of 500+ cities across the world of varying distance from the equator. To accomplish this, you'll be utilizing a simple Python library, the OpenWeatherMap API, and a little common sense to create a representative model of weather across world cities.
Your objective is to build a series of scatter plots to showcase the following relationships:
* Temperature (F) vs. Latitude
* Humidity (%) vs. Latitude
* Cloudiness (%) vs. Latitude
* Wind Speed (mph) vs. Latitude

In [1]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from citipy import citipy
from pprint import pprint
from config import api_key

In [2]:
# import coordinates
coordinates = pd.read_excel('lat_long_generator.xlsx', sheetname='Sheet1', header=None)
coordinates = coordinates.rename(columns={0:'Latitude', 1:'Longitude'})


In [3]:
# find closest city
coordinates["City"] = ""

for index,row in coordinates.iterrows():
    city = citipy.nearest_city(row["Latitude"],row["Longitude"])
    coordinates.set_value(index,"City",city.city_name)
coordinates.head()

,Latitude,Longitude,City
0,-50.765508,141.617104,new norfolk
1,44.569233,-46.525207,torbay
2,-88.939777,153.293126,bluff
3,51.112213,-137.340710,sitka
4,60.369664,-97.968504,thompson


In [8]:
# Look for duplicates and remove when found
sort_dupes = coordinates.sort_values('City', ascending=False)
dedupe_cities = coordinates.drop_duplicates('City').sort_index()

# Reduce list to 500
city_final = dedupe_cities.sample(500)
city = city_final['City']
city_df = pd.DataFrame(city)

,City
2107,pacific grove
153,saleaula
654,phangnga
1245,lastoursville
2109,opobo


In [10]:
# Save config information and define variables
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"

query_url = f"{url}appid={api_key}&q={city}&unit={units}"

for index,row in city_df.iterrows():
    city_wx_response = requests.get(query_url)
    city_wx = city_wx_response.json()
    city_final.set_value(index,"Latitude",city_wx.get("coord",{}).get("lat"))
    city_final.set_value(index,"Longitude",city_wx.get("coord",{}).get("lon"))
    city_final.set_value(index,"Temperature",city_wx.get("main",{}).get("temp_max"))
    city_final.set_value(index,"Humidity",city_wx.get("main",{}).get("humidity"))
    city_final.set_value(index,"Cloudiness",city_wx.get("clouds",{}).get("all"))
    city_final.set_value(index,"Wind speed",city_wx.get("wind",{}).get("speed"))


JSONDecodeError: Expecting value: line 1 column 1 (char 0)